In [1]:
import torch

num_entries=5000
db =torch.rand(num_entries ) >0.5
db

tensor([0, 0, 1,  ..., 0, 0, 0], dtype=torch.uint8)

In [2]:
db.shape



torch.Size([5000])

In [3]:
db[0:5]

tensor([0, 0, 1, 1, 0], dtype=torch.uint8)

In [4]:
remove_index=2

In [5]:
db[0:2]

tensor([0, 0], dtype=torch.uint8)

In [6]:
db[3:5]

tensor([1, 0], dtype=torch.uint8)

In [7]:
def get_parallel_db(db,remove_index):
    return torch.cat((db[0:remove_index],db[remove_index+1:]))
    



In [8]:
get_parallel_db(db,remove_index).shape

torch.Size([4999])

In [9]:
def get_parallel_dbs(db):
    parallel_dbs=list()
    for i in range(len(db)):
        pdb= get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs    


In [10]:
def create_db_and_parallelize(num_entries):
    db=torch.rand(num_entries)>0.5
    pdbs=get_parallel_dbs(db)
    
    return db,pdbs
    

In [11]:
db, pdbs= create_db_and_parallelize(20)

In [12]:
db

tensor([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
       dtype=torch.uint8)

In [13]:
pdbs

[tensor([1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,

In [14]:
def query(db):
    return db.sum()

In [15]:
full_query_result=query(db)

In [16]:
query(pdbs[15])

tensor(12)

In [17]:
query(pdbs[1])

tensor(11)

In [18]:
max_distance=0 
for pdb in pdbs:
    pdb_result=query(pdb)
    
    db_distance = torch.abs(full_query_result-pdb_result)
    if (db_distance>max_distance):
        max_distance= db_distance

In [19]:
max_distance

tensor(1)

In [23]:
def senstivity(query, n_entries=1000):
    
    db, pdbs= create_db_and_parallelize(n_entries) 

    full_query_result=query(db)


    max_distance=0 
    for pdb in pdbs:
        pdb_result=query(pdb)
    
    db_distance = torch.abs(full_query_result-pdb_result)
    if (db_distance>max_distance):
        max_distance= db_distance
    return max_distance    
    



In [24]:
def query(db):
    return db.float().mean()

In [25]:
senstivity(query)

tensor(0.0005)

In [72]:
def query(db,threshold=5):
     return (db.sum()>threshold).float()
       
db,pdbs= create_db_and_parallelize(10)

In [73]:
db.sum()


tensor(7)

In [74]:
senstivity(query,10)

0

In [75]:
for i in range(10):
    sens_f= senstivity(query,n_entries=10)
    print(sens_f)
    

0
0
0
0
tensor(1.)
0
0
0
0
0


In [78]:
db,_=create_db_and_parallelize(num_entries=100)
pdb=get_parallel_db(db,remove_index=10)

In [90]:
db[10]
pdb[10]

tensor(0, dtype=torch.uint8)

In [91]:
sum(db)

tensor(46, dtype=torch.uint8)

In [86]:
#differencing attacks
sum(db)-sum(pdb)

tensor(1, dtype=torch.uint8)

In [92]:
(sum(db).float()>45)-(sum(pdb).float()>49)

tensor(1, dtype=torch.uint8)